In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))

In [7]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [8]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,1.0,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2],NaN
1,JANUARY,13.0,The White Tiger,Netflix,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
2,JANUARY,14.0,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[3],NaN
3,JANUARY,15.0,The Dig,Netflix / Clerkenwell Films,Simon Stone (director); Moira Buffini (screenp...,[4],NaN
4,JANUARY,15.0,Outside the Wire,Netflix,"Mikael Håfström (director); Rob Yescombe, Rowa...",[5],NaN
...,...,...,...,...,...,...,...
356,DECEMBER,25.0,The Tragedy of Macbeth,Apple TV+ / A24 / IAC Films,Joel Coen (director/screenplay); Denzel Washin...,NaN,[273]
357,DECEMBER,25.0,A Journal for Jordan,Columbia Pictures / Escape Artists / Bron Studios,Denzel Washington (director); Virgil Williams ...,NaN,[274]
358,DECEMBER,25.0,American Underdog,Lionsgate,"Erwin brothers (directors); Jon Erwin, David A...",NaN,[275]
359,DECEMBER,26.0,Memoria,Neon,Apichatpong Weerasethakul (director/acreenplay...,NaN,[276]


In [9]:
df_2021 = df[['Title']]

In [10]:
!pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from tmdbv3api import TMDb
import json
tmdb = TMDb()
tmdb.api_key = '2d71ac9f8f76e2fe996cdc92cab9f9cd'

In [12]:
from tmdbv3api import Movie
import json
tmdb_movie = Movie() 

In [13]:
def ids(x):
  result = tmdb_movie.search(x)
  if not result:
    return np.NaN
  else:
    return int(result[0].id)

In [14]:
df_2021['id'] = df_2021['Title'].map(lambda x: ids(str(x)))

<ipython-input-14-1e7fcbae526d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['id'] = df_2021['Title'].map(lambda x: ids(str(x)))


In [15]:
df_2021

,Title,id
0,Shadow in the Cloud,675327
1,The White Tiger,628534
2,Locked Down,741228
3,The Dig,532865
4,Outside the Wire,775996
...,...,...
356,The Tragedy of Macbeth,591538
357,A Journal for Jordan,501841
358,American Underdog,673309
359,Memoria,511819


In [16]:
df_2021[df_2021['id'].isnull()]

,Title,id


In [17]:
df_2021['id'] = df_2021['id'].astype('int')

<ipython-input-17-152f78e33417>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['id'] = df_2021['id'].astype('int')


In [18]:
def genres(movie_id):
    genres=[]
  
    movie_details = tmdb_movie.details(movie_id)
    movie_genres = movie_details.genres
    if movie_genres:
          genre_str = " " 
          for i in range(0,len(movie_genres)):
              genres.append(movie_genres[i]['name'])
          return genre_str.join(genres)
    else:
          return np.NaN


In [19]:
df_2021['genres'] = df_2021['id'].map(lambda x: genres(int(x)))

<ipython-input-19-bd699cdb4128>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['genres'] = df_2021['id'].map(lambda x: genres(int(x)))


In [20]:
df_2021

,Title,id,genres
0,Shadow in the Cloud,675327,Horror Action War
1,The White Tiger,628534,Drama
2,Locked Down,741228,Comedy Crime Drama
3,The Dig,532865,Drama History
4,Outside the Wire,775996,Thriller Action Science Fiction
...,...,...,...
356,The Tragedy of Macbeth,591538,Drama War
357,A Journal for Jordan,501841,Drama Romance
358,American Underdog,673309,Drama
359,Memoria,511819,Drama Science Fiction Mystery


In [21]:
def keywords(movie_id):
    keywords=[]

    movie_keywords = tmdb_movie.keywords(movie_id)
    #print(movie_keywords)
    if movie_keywords:
          #print(movie_keywords['keywords'])
          return movie_keywords['keywords']
    else:
          return np.NaN

In [22]:
df_2021['keywords'] = df_2021['id'].map(lambda x: keywords(int(x)))

<ipython-input-22-0d0bd01e1317>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['keywords'] = df_2021['id'].map(lambda x: keywords(int(x)))


In [23]:
df_2021

,Title,id,genres,keywords
0,Shadow in the Cloud,675327,Horror Action War,"[[id, name], [id, name], [id, name], [id, name..."
1,The White Tiger,628534,Drama,"[[id, name], [id, name], [id, name], [id, name..."
2,Locked Down,741228,Comedy Crime Drama,"[[id, name], [id, name], [id, name], [id, name..."
3,The Dig,532865,Drama History,"[[id, name], [id, name], [id, name], [id, name..."
4,Outside the Wire,775996,Thriller Action Science Fiction,"[[id, name], [id, name], [id, name], [id, name..."
...,...,...,...,...
356,The Tragedy of Macbeth,591538,Drama War,"[[id, name], [id, name], [id, name], [id, name..."
357,A Journal for Jordan,501841,Drama Romance,[]
358,American Underdog,673309,Drama,"[[id, name], [id, name], [id, name], [id, name..."
359,Memoria,511819,Drama Science Fiction Mystery,"[[id, name], [id, name], [id, name]]"


In [24]:
df_2021['keywords'] = df_2021['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

<ipython-input-24-710d9a28893a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['keywords'] = df_2021['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [25]:
s = df_2021.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

<ipython-input-25-65c8c13ccd47>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df_2021.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


based on novel or book    38
based on true story       23
woman director            21
sequel                    20
murder                    18
Name: keyword, dtype: int64

In [26]:
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

In [27]:
s = s[s > 1]
stemmer = SnowballStemmer('english')

In [28]:
def keywordsFiltered(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [29]:
df_2021['keywords'] = df_2021['keywords'].apply(keywordsFiltered)
df_2021['keywords'] = df_2021['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

<ipython-input-29-746c4e5c3677>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['keywords'] = df_2021['keywords'].apply(keywordsFiltered)
<ipython-input-29-746c4e5c3677>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['keywords'] = df_2021['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])


In [30]:
df_2021

,Title,id,genres,keywords
0,Shadow in the Cloud,675327,Horror Action War,"[world war ii, woman director]"
1,The White Tiger,628534,Drama,"[based on novel or book, rags to rich, class d..."
2,Locked Down,741228,Comedy Crime Drama,"[london, england, lockdown, pandem, covid-19]"
3,The Dig,532865,Drama History,"[based on novel or book, based on true stori, ..."
4,Outside the Wire,775996,Thriller Action Science Fiction,"[android, killer robot, near futur]"
...,...,...,...,...
356,The Tragedy of Macbeth,591538,Drama War,"[husband wife relationship, based on play or m..."
357,A Journal for Jordan,501841,Drama Romance,[]
358,American Underdog,673309,Drama,"[american footbal, biographi, los angeles, cal..."
359,Memoria,511819,Drama Science Fiction Mystery,[colombia]


In [31]:
df_2021['keywords'] = df_2021['keywords'].map(lambda x: " ".join(x))
df_2021['keywords']

<ipython-input-31-91344671b26c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['keywords'] = df_2021['keywords'].map(lambda x: " ".join(x))


0                            world war ii woman director
1       based on novel or book rags to rich class differ
2               london, england lockdown pandem covid-19
3      based on novel or book based on true stori cla...
4                        android killer robot near futur
                             ...                        
356    husband wife relationship based on play or mus...
357                                                     
358    american footbal biographi los angeles, califo...
359                                             colombia
360    danc danc hors road trip flashback family rela...
Name: keywords, Length: 361, dtype: object

In [32]:
def credits(movie_id):
    cast=[]
  
    movie_credits = tmdb_movie.credits(movie_id)
    if movie_credits['cast'] and movie_credits['crew']:
            for i in movie_credits['crew']:
                if i.get('job') == 'Director':
                    cast.append(i.get('name'))
                    break
            for i in range(0, len(movie_credits['cast'])):
                cast.append(movie_credits['cast'][i]['name'])
            return cast
    else:
            return np.NaN

In [33]:
df_2021['cast'] = df_2021['id'].map(lambda x: credits(int(x)))

<ipython-input-33-ac5a5741e6bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['cast'] = df_2021['id'].map(lambda x: credits(int(x)))


In [34]:
df_2021

,Title,id,genres,keywords,cast
0,Shadow in the Cloud,675327,Horror Action War,world war ii woman director,"[Roseanne Liang, Chloë Grace Moretz, Nick Robi..."
1,The White Tiger,628534,Drama,based on novel or book rags to rich class differ,"[Ramin Bahrani, Adarsh Gourav, Rajkummar Rao, ..."
2,Locked Down,741228,Comedy Crime Drama,"london, england lockdown pandem covid-19","[Doug Liman, Anne Hathaway, Chiwetel Ejiofor, ..."
3,The Dig,532865,Drama History,based on novel or book based on true stori cla...,"[Simon Stone, Carey Mulligan, Ralph Fiennes, L..."
4,Outside the Wire,775996,Thriller Action Science Fiction,android killer robot near futur,"[Mikael Håfström, Anthony Mackie, Damson Idris..."
...,...,...,...,...,...
356,The Tragedy of Macbeth,591538,Drama War,husband wife relationship based on play or mus...,"[Joel Coen, Denzel Washington, Frances McDorma..."
357,A Journal for Jordan,501841,Drama Romance,,"[Denzel Washington, Chanté Adams, Michael B. J..."
358,American Underdog,673309,Drama,"american footbal biographi los angeles, califo...","[Jon Erwin, Zachary Levi, Anna Paquin, Hayden ..."
359,Memoria,511819,Drama Science Fiction Mystery,colombia,"[Apichatpong Weerasethakul, Tilda Swinton, Agn..."


In [35]:
df_2021['cast'] = df_2021['cast'].apply(lambda x: [i for i in x] if isinstance(x, list) else [])
df_2021['cast'] = df_2021['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

<ipython-input-35-29b831f7c039>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['cast'] = df_2021['cast'].apply(lambda x: [i for i in x] if isinstance(x, list) else [])
<ipython-input-35-29b831f7c039>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['cast'] = df_2021['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])


In [36]:
df_2021

,Title,id,genres,keywords,cast
0,Shadow in the Cloud,675327,Horror Action War,world war ii woman director,"[roseanneliang, chloëgracemoretz, nickrobinson..."
1,The White Tiger,628534,Drama,based on novel or book rags to rich class differ,"[raminbahrani, adarshgourav, rajkummarrao, pri..."
2,Locked Down,741228,Comedy Crime Drama,"london, england lockdown pandem covid-19","[dougliman, annehathaway, chiwetelejiofor, ste..."
3,The Dig,532865,Drama History,based on novel or book based on true stori cla...,"[simonstone, careymulligan, ralphfiennes, lily..."
4,Outside the Wire,775996,Thriller Action Science Fiction,android killer robot near futur,"[mikaelhåfström, anthonymackie, damsonidris, e..."
...,...,...,...,...,...
356,The Tragedy of Macbeth,591538,Drama War,husband wife relationship based on play or mus...,"[joelcoen, denzelwashington, francesmcdormand,..."
357,A Journal for Jordan,501841,Drama Romance,,"[denzelwashington, chantéadams, michaelb.jorda..."
358,American Underdog,673309,Drama,"american footbal biographi los angeles, califo...","[jonerwin, zacharylevi, annapaquin, haydenzall..."
359,Memoria,511819,Drama Science Fiction Mystery,colombia,"[apichatpongweerasethakul, tildaswinton, agnes..."


In [37]:
df_2021['director'] = df_2021['cast'].map(lambda x:x[0] if len(x) >=1 else x)

<ipython-input-37-ea91dbc1b54b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['director'] = df_2021['cast'].map(lambda x:x[0] if len(x) >=1 else x)


In [38]:
df_2021

,Title,id,genres,keywords,cast,director
0,Shadow in the Cloud,675327,Horror Action War,world war ii woman director,"[roseanneliang, chloëgracemoretz, nickrobinson...",roseanneliang
1,The White Tiger,628534,Drama,based on novel or book rags to rich class differ,"[raminbahrani, adarshgourav, rajkummarrao, pri...",raminbahrani
2,Locked Down,741228,Comedy Crime Drama,"london, england lockdown pandem covid-19","[dougliman, annehathaway, chiwetelejiofor, ste...",dougliman
3,The Dig,532865,Drama History,based on novel or book based on true stori cla...,"[simonstone, careymulligan, ralphfiennes, lily...",simonstone
4,Outside the Wire,775996,Thriller Action Science Fiction,android killer robot near futur,"[mikaelhåfström, anthonymackie, damsonidris, e...",mikaelhåfström
...,...,...,...,...,...,...
356,The Tragedy of Macbeth,591538,Drama War,husband wife relationship based on play or mus...,"[joelcoen, denzelwashington, francesmcdormand,...",joelcoen
357,A Journal for Jordan,501841,Drama Romance,,"[denzelwashington, chantéadams, michaelb.jorda...",denzelwashington
358,American Underdog,673309,Drama,"american footbal biographi los angeles, califo...","[jonerwin, zacharylevi, annapaquin, haydenzall...",jonerwin
359,Memoria,511819,Drama Science Fiction Mystery,colombia,"[apichatpongweerasethakul, tildaswinton, agnes...",apichatpongweerasethakul


In [39]:
df_2021['actor1']=df_2021['cast'].map(lambda x:x[1] if len(x) >=2 else x)
df_2021['actor2']=df_2021['cast'].map(lambda x:x[2] if len(x) >=3 else x)
df_2021['actor3']=df_2021['cast'].map(lambda x:x[3] if len(x) >=4 else x)

In [40]:
df_2021

,Title,id,genres,keywords,cast,director,actor1,actor2,actor3
0,Shadow in the Cloud,675327,Horror Action War,world war ii woman director,"[roseanneliang, chloëgracemoretz, nickrobinson...",roseanneliang,chloëgracemoretz,nickrobinson,beulahkoale
1,The White Tiger,628534,Drama,based on novel or book rags to rich class differ,"[raminbahrani, adarshgourav, rajkummarrao, pri...",raminbahrani,adarshgourav,rajkummarrao,priyankachoprajonas
2,Locked Down,741228,Comedy Crime Drama,"london, england lockdown pandem covid-19","[dougliman, annehathaway, chiwetelejiofor, ste...",dougliman,annehathaway,chiwetelejiofor,stephenmerchant
3,The Dig,532865,Drama History,based on novel or book based on true stori cla...,"[simonstone, careymulligan, ralphfiennes, lily...",simonstone,careymulligan,ralphfiennes,lilyjames
4,Outside the Wire,775996,Thriller Action Science Fiction,android killer robot near futur,"[mikaelhåfström, anthonymackie, damsonidris, e...",mikaelhåfström,anthonymackie,damsonidris,enzocilenti
...,...,...,...,...,...,...,...,...,...
356,The Tragedy of Macbeth,591538,Drama War,husband wife relationship based on play or mus...,"[joelcoen, denzelwashington, francesmcdormand,...",joelcoen,denzelwashington,francesmcdormand,coreyhawkins
357,A Journal for Jordan,501841,Drama Romance,,"[denzelwashington, chantéadams, michaelb.jorda...",denzelwashington,chantéadams,michaelb.jordan,jalonchristian
358,American Underdog,673309,Drama,"american footbal biographi los angeles, califo...","[jonerwin, zacharylevi, annapaquin, haydenzall...",jonerwin,zacharylevi,annapaquin,haydenzaller
359,Memoria,511819,Drama Science Fiction Mystery,colombia,"[apichatpongweerasethakul, tildaswinton, agnes...",apichatpongweerasethakul,tildaswinton,agnesbrekke,danielgiménezcacho


In [41]:
df_2021= df_2021.loc[:,['id','director','actor1','actor2','actor3','genres','keywords','Title']]

In [42]:
df_2021 = df_2021.rename(columns={'Title':'title'})

In [43]:
df_2021['title'] = df_2021['title'].str.lower()
df_2021['actor1'] = df_2021['actor1'].str.lower()
df_2021['actor2'] = df_2021['actor2'].str.lower()
df_2021['actor3'] = df_2021['actor3'].str.lower()
# data['director'] = data['director'].str.lower()
df_2021['comb'] = df_2021['genres'] + ' ' + df_2021['keywords'] + ' '  + df_2021['actor1'] + ' '  + df_2021['actor2'] + ' '  + df_2021['actor3'] + ' ' + df_2021['director']

In [44]:
df_2021

,id,director,actor1,actor2,actor3,genres,keywords,title,comb
0,675327,roseanneliang,chloëgracemoretz,nickrobinson,beulahkoale,Horror Action War,world war ii woman director,shadow in the cloud,Horror Action War world war ii woman director ...
1,628534,raminbahrani,adarshgourav,rajkummarrao,priyankachoprajonas,Drama,based on novel or book rags to rich class differ,the white tiger,Drama based on novel or book rags to rich clas...
2,741228,dougliman,annehathaway,chiwetelejiofor,stephenmerchant,Comedy Crime Drama,"london, england lockdown pandem covid-19",locked down,"Comedy Crime Drama london, england lockdown pa..."
3,532865,simonstone,careymulligan,ralphfiennes,lilyjames,Drama History,based on novel or book based on true stori cla...,the dig,Drama History based on novel or book based on ...
4,775996,mikaelhåfström,anthonymackie,damsonidris,enzocilenti,Thriller Action Science Fiction,android killer robot near futur,outside the wire,Thriller Action Science Fiction android killer...
...,...,...,...,...,...,...,...,...,...
356,591538,joelcoen,denzelwashington,francesmcdormand,coreyhawkins,Drama War,husband wife relationship based on play or mus...,the tragedy of macbeth,Drama War husband wife relationship based on p...
357,501841,denzelwashington,chantéadams,michaelb.jordan,jalonchristian,Drama Romance,,a journal for jordan,Drama Romance chantéadams michaelb.jordan jal...
358,673309,jonerwin,zacharylevi,annapaquin,haydenzaller,Drama,"american footbal biographi los angeles, califo...",american underdog,"Drama american footbal biographi los angeles, ..."
359,511819,apichatpongweerasethakul,tildaswinton,agnesbrekke,danielgiménezcacho,Drama Science Fiction Mystery,colombia,memoria,Drama Science Fiction Mystery colombia tildasw...


In [45]:
df_2021.to_csv('movies2021.csv',index=False)

In [46]:
from google.colab import files
files.download("movies2021.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>